## argsort

Collaborative filtering 의 코드를 완성하기 위한 몇 가지 numpy handling 들을 배워봅니다.

In [1]:
import numpy as np

np.random.seed(0)
np.set_printoptions(precision=4, suppress=True)

x = np.random.random_sample((10,5))
x

array([[0.5488, 0.7152, 0.6028, 0.5449, 0.4237],
       [0.6459, 0.4376, 0.8918, 0.9637, 0.3834],
       [0.7917, 0.5289, 0.568 , 0.9256, 0.071 ],
       [0.0871, 0.0202, 0.8326, 0.7782, 0.87  ],
       [0.9786, 0.7992, 0.4615, 0.7805, 0.1183],
       [0.6399, 0.1434, 0.9447, 0.5218, 0.4147],
       [0.2646, 0.7742, 0.4562, 0.5684, 0.0188],
       [0.6176, 0.6121, 0.6169, 0.9437, 0.6818],
       [0.3595, 0.437 , 0.6976, 0.0602, 0.6668],
       [0.6706, 0.2104, 0.1289, 0.3154, 0.3637]])

`x` 는 (10, 5) 크기의 행렬입니다. 각 row 별로 column 의 값들의 순서를 정렬하려면 `argsort(axis=1)` 을 이용합니다. 반대로 각 column 별로 row 의 값들의 순서를 정렬하려면 `argsort(axis=0)` 을 이용합니다.

In [2]:
topk = 2
x.argsort(axis=1)

array([[4, 3, 0, 2, 1],
       [4, 1, 0, 2, 3],
       [4, 1, 2, 0, 3],
       [1, 0, 3, 2, 4],
       [4, 2, 3, 1, 0],
       [1, 4, 3, 0, 2],
       [4, 0, 2, 3, 1],
       [1, 2, 0, 4, 3],
       [3, 0, 1, 4, 2],
       [2, 1, 3, 4, 0]])

정렬된 값의 모든 rows 에 대하여 앞의 topk 개의 columns 만 가져오면 각 rows 별로 가장 가까운 topk 개의 column indices 를 가져올 수 있습니다.

In [3]:
x.argsort(axis=1)[:,:topk]

array([[4, 3],
       [4, 1],
       [4, 1],
       [1, 0],
       [4, 2],
       [1, 4],
       [4, 0],
       [1, 2],
       [3, 0],
       [2, 1]])

이 행렬에 `flatten()` 함수를 실행하면 각 rows 가 하나씩 차례로 concatenation 이 된 형태의 array 가 만들어집니다.

In [4]:
cols = x.argsort(axis=1)[:,:topk].flatten()
cols

array([4, 3, 4, 1, 4, 1, 1, 0, 4, 2, 1, 4, 4, 0, 1, 2, 3, 0, 2, 1])

위의 값은 column indices 의 값이므로, 각 값에 해당하는 row indices 도 만들어봅니다. 이는 `numpy.repeat()` 와 `numpy.arange()` 함수를 이용하여 만들 수 있습니다.

In [5]:
rows = np.repeat(np.arange(10), repeats=topk)
rows

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9])

numpy.ndarray 를 slicing 하는 방법으로 row, column indices 를 각각 array 로 만들어 입력할 수도 있습니다. 위의 `x` 와 slicing 된 값을 직접 비교해 보시기 바랍니다.

In [6]:
x[rows,cols]

array([0.4237, 0.5449, 0.3834, 0.4376, 0.071 , 0.5289, 0.0202, 0.0871,
       0.1183, 0.4615, 0.1434, 0.4147, 0.0188, 0.2646, 0.6121, 0.6169,
       0.0602, 0.3595, 0.1289, 0.2104])

그런데 위의 slicing 된 결과 역시 array 입니다. 이를 다시 (10, topk) 형식의 행렬로 변환하기 위하여 `reshape()` 함수를 이용합니다.

In [7]:
x[rows,cols].reshape(-1,topk)

array([[0.4237, 0.5449],
       [0.3834, 0.4376],
       [0.071 , 0.5289],
       [0.0202, 0.0871],
       [0.1183, 0.4615],
       [0.1434, 0.4147],
       [0.0188, 0.2646],
       [0.6121, 0.6169],
       [0.0602, 0.3595],
       [0.1289, 0.2104]])

## distance computation

위의 numpy.ndarray slicing 은 k argmin distance 를 계산하기 위해 필요합니다. Scikit-learn 에서는 `pairwise_distances`, `argmin_pairwise_distances` 등의 함수를 제공해 주지만 k argmin 함수를 제공하지 않습니다. 이는 직접 만들어야 합니다. `pairwise_distances(A, B, metric=metric)` 는 A 의 각 rows 에 대하여 B 의 각 rows 간의 metric distance 를 계산합니다. 그러므로 A, B 의 shape 은 각각 (n,k), (m,k) 여야 합니다.

In [8]:
from sklearn.metrics import pairwise_distances

dist = pairwise_distances(x[:2,:], x, metric='euclidean')
dist.shape

(2, 10)

In [9]:
dist

array([[0.    , 0.5891, 0.6035, 1.0013, 0.6004, 0.6729, 0.5199, 0.4912,
        0.6451, 0.7419],
       [0.5891, 0.    , 0.4832, 0.8724, 0.7283, 0.5344, 0.8589, 0.443 ,
        1.0081, 1.027 ]])

앞서 각 row 별로 가장 작은 값을 지닌 colum indices 를 slicing 하는 방법을 살펴보았습니다.

In [10]:
topk = 3
indices = dist.argsort(axis=1)[:,:topk]
indices.shape

(2, 3)

In [11]:
indices

array([[0, 7, 6],
       [1, 7, 2]])

In [12]:
dist

array([[0.    , 0.5891, 0.6035, 1.0013, 0.6004, 0.6729, 0.5199, 0.4912,
        0.6451, 0.7419],
       [0.5891, 0.    , 0.4832, 0.8724, 0.7283, 0.5344, 0.8589, 0.443 ,
        1.0081, 1.027 ]])

그리고 그 indices 에 해당하는 값도 모두 가져오는 방법도 연습하였습니다.

In [13]:
cols = dist.argsort(axis=1)[:,:topk].flatten()
rows = np.repeat(np.arange(dist.shape[0]), repeats=topk)
dist[rows, cols].reshape(-1,topk)

array([[0.    , 0.4912, 0.5199],
       [0.    , 0.443 , 0.4832]])

## matrix inner product

벡터의 내적을 하는 방법에 대해서도 살펴봅니다. 행렬 곲은 정확히는 matrix multiplication, element-wise product, 등으로 나뉘어집니다.

In [14]:
a = np.random.randint(0, 3, size=(1, 3)) # similar sim
b = np.random.randint(0, 3, size=(3, 5)) # candidate history

print(a)
print(b)

[[1 2 0]]
[[0 2 0 0 0]
 [0 0 0 2 0]
 [2 1 1 1 0]]


`numpy.dot()` 함수는 matrix multiplication 을 수행합니다.

In [15]:
c = np.dot(a, b)
print(c)
print(c.shape)

[[0 2 0 4 0]]
(1, 5)


## sparse matrix nonzero element indices

Sparse matrix 는 대부분의 값이 0 일 경우, 0 이 아닌 값의 row, column indices 와 그에 해당하는 값만을 저장하는 방식입니다. 이는 다양한 종류가 있으며, 자세한 설명은 아래의 포스트를 참고하세요.

https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/

Sparse matrix 에서 0 이 아닌 값의 위치를 알려면 `nonzero()` 함수를 실행합니다. 그 결과는 각각 row, column indices array 가 출력됩니다.

In [16]:
from scipy.sparse import csr_matrix

s = csr_matrix(np.asarray([
    [0, 0, 1, 0, 0, 2],
    [0, 0, 0, 0, 3, 0],
    [0, 4, 0, 0, 0, 0]
]))
rows, cols = s.nonzero()
print(rows)
print(cols)

[0 0 1 2]
[2 5 4 1]


만약 한 row 의 nonzero elements 의 indices 를 알려면 sparse matrix 에 slicing 을 한 결과에 `nonzero()` 함수를 실행하면 됩니다.

In [17]:
s[0].nonzero()

(array([0, 0], dtype=int32), array([2, 5], dtype=int32))

0 이 아닌 실제 값들은 `data` array 에 저장되어 있습니다.

In [18]:
s.data

array([1, 2, 3, 4], dtype=int64)